# Portfolio Algorithm Backtesting

Import necessary packages

In [16]:
import pandas as pd
import numpy as np
import yfinance as yf

Assign variables

In [17]:
td_year = 252
rfr = 0.035

Assign market weights (update to pull live figures)

In [18]:
asset_data = [
    {
        "Asset Class": "US Equities",
        "Index": "Vanguard Total Stock Market Index Fund",
        "Ticker": "VTI",
        "Weight": 0.18
    },
    {
        "Asset Class": "Foreign Developed Equities",
        "Index": "MSCI EAFE Index",
        "Ticker": "EFA",
        "Weight": 0.13
    },
    {
        "Asset Class": "Foreign Emerging Equities",
        "Index": "MSCI Emerging Markets Index",
        "Ticker": "MME=F",
        "Weight": 0.05
    },
    {
        "Asset Class": "Global Corporate Bonds",
        "Index": "Vanguard Total Bond Market Index Fund",
        "Ticker": "BND",
        "Weight": 0.20
    },
    {
        "Asset Class": "Thirty-Year Bonds",
        "Index": "Vanguard Total International Bond Index Fund",
        "Ticker": "BNDX",
        "Weight": 0.28
    },
    {
        "Asset Class": "TIPs",
        "Index": "iShares TIPS Bond ETF",
        "Ticker": "TIP",
        "Weight": 0.02
    },
    {
        "Asset Class": "REITs",
        "Index": "FTSE NAREIT All Equity REITs Index",
        "Ticker": "^FNER",
        "Weight": 0.05
    },
    {
        "Asset Class": "Commodities",
        "Index": "Invesco DB Commodity Index Tracking Fund",
        "Ticker": "DBC",
        "Weight": 0.05
    },
    {
        "Asset Class": "Gold",
        "Index": "SPDR Gold Shares",
        "Ticker": "GLD",
        "Weight": 0.05
    }
]


portfolio = pd.DataFrame(asset_data)
portfolio = portfolio.set_index("Asset Class")

asset_tickers = portfolio[portfolio["Ticker"] != ""].Ticker.tolist()

portfolio

,Index,Ticker,Weight
Asset Class,,,
US Equities,Vanguard Total Stock Market Index Fund,VTI,0.18
Foreign Developed Equities,MSCI EAFE Index,EFA,0.13
Foreign Emerging Equities,MSCI Emerging Markets Index,MME=F,0.05
Global Corporate Bonds,Vanguard Total Bond Market Index Fund,BND,0.20
Thirty-Year Bonds,Vanguard Total International Bond Index Fund,BNDX,0.28
TIPs,iShares TIPS Bond ETF,TIP,0.02
REITs,FTSE NAREIT All Equity REITs Index,^FNER,0.05
Commodities,Invesco DB Commodity Index Tracking Fund,DBC,0.05
Gold,SPDR Gold Shares,GLD,0.05


Calculate annual risk and returns for each portfolio asset

In [19]:
df = yf.download(asset_tickers)
returns = np.log(df["Adj Close"].div(df["Adj Close"].shift(1))).dropna()

for ticker in asset_tickers:
    
    portfolio.loc[portfolio["Ticker"] == ticker, "ann. Risk"] = returns[ticker].std() * np.sqrt(td_year)
    portfolio.loc[portfolio["Ticker"] == ticker, "CAGR"] = np.exp(returns[ticker].mean() * td_year) - 1
    portfolio.loc[portfolio["Ticker"] == ticker, "RaR"] = portfolio["CAGR"] / portfolio["ann. Risk"]   

portfolio

[*********************100%***********************]  9 of 9 completed


,Index,Ticker,Weight,ann. Risk,CAGR,RaR
Asset Class,,,,,,
US Equities,Vanguard Total Stock Market Index Fund,VTI,0.18,0.186474,0.105749,0.567099
Foreign Developed Equities,MSCI EAFE Index,EFA,0.13,0.179421,0.040537,0.225929
Foreign Emerging Equities,MSCI Emerging Markets Index,MME=F,0.05,0.217205,0.008193,0.037722
Global Corporate Bonds,Vanguard Total Bond Market Index Fund,BND,0.20,0.052731,0.021662,0.410792
Thirty-Year Bonds,Vanguard Total International Bond Index Fund,BNDX,0.28,0.039413,0.023640,0.599817
TIPs,iShares TIPS Bond ETF,TIP,0.02,0.059493,0.026878,0.451790
REITs,FTSE NAREIT All Equity REITs Index,^FNER,0.05,0.212597,0.024575,0.115595
Commodities,Invesco DB Commodity Index Tracking Fund,DBC,0.05,0.179277,0.017444,0.097301
Gold,SPDR Gold Shares,GLD,0.05,0.142337,0.052908,0.371712


In [20]:
duration = returns.index[-1] - returns.index[0]
years = duration.total_seconds() / (365.25 * 24 * 60 * 60)
weights = portfolio.Weight.values
cov_matrix = returns.cov() * td_year

In [21]:
portfolioreturns = portfolio.CAGR.dot(weights)
portfoliorisk =  np.sqrt((weights.dot(cov_matrix).dot(weights)))
roll_max = df["Adj Close"].dot(weights).cummax()
daily_drawdown = df["Adj Close"].dot(weights)/roll_max - 1.0
max_daily_drawdown = daily_drawdown.cummin().min()

In [22]:
summary = pd.DataFrame()
summary.loc["GMP","Years"] = years
summary.loc["GMP","Returns"] = portfolioreturns
summary.loc["GMP","Volatility"] = portfoliorisk
summary.loc["GMP","Sharpe"] = (portfolioreturns - rfr) / portfoliorisk
summary.loc["GMP","Max Drawdown"] = max_daily_drawdown
summary

,Years,Returns,Volatility,Sharpe,Max Drawdown
GMP,9.122519,0.04095,0.081971,0.072584,-0.224282


In [23]:
df = yf.download("^GSPC", start = returns.index[0])
sp500 = np.log(df["Adj Close"].div(df["Adj Close"].shift(1))).dropna()

sp500returns = np.exp(sp500.mean() * td_year) - 1
sp500risk = sp500.std() * np.sqrt(td_year)
sp500roll_max = df['Adj Close'].cummax()
sp500daily_drawdown = df['Adj Close']/sp500roll_max - 1.0
sp500max_daily_drawdown = sp500daily_drawdown.cummin().min()

summary.loc["S&P 500","Years"] = years
summary.loc["S&P 500","Returns"] = sp500returns
summary.loc["S&P 500","Volatility"] = sp500risk
summary.loc["S&P 500","Sharpe"] = (sp500returns - rfr) / sp500risk
summary.loc["S&P 500","Max Drawdown"] = sp500max_daily_drawdown
summary

[*********************100%***********************]  1 of 1 completed


,Years,Returns,Volatility,Sharpe,Max Drawdown
GMP,9.122519,0.040950,0.081971,0.072584,-0.224282
S&P 500,9.122519,0.096112,0.182933,0.334067,-0.339250


In [24]:
summary.to_csv("PortfolioSummary.csv")